In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive

/content/drive/MyDrive


In [2]:
ls

drive/  sample_data/


In [3]:
cd transformer_input

[Errno 2] No such file or directory: 'transformer_input'
/content


In [4]:
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 53.0 MB/s eta 0:00:00


In [5]:
ls /content/drive/MyDrive/

'Colab Notebooks'/     'model (9).h5'
 DNN02.ipynb            model.h5
 EVA4Phase2/            model_weights_img_aug_120_127_op.h5
'Getting started.pdf'   model_weights_img_aug_30_60.h5
 IMDB/                  model_weights_img_aug_60_90.h5
'model (1).h5'          model_weights_img_aug_90_120.h5
'model (2).h5'          model_weights_img_aug_90_120_op.h5
'model (3).h5'          model_weights_img_aug.h5
'model (4).h5'          model_weights_img_aug_image_man_0_30.h5
'model (5).h5'          transformer_input/
'model (6).h5'          Untitled
'model (7).h5'          Untitled0.ipynb
'model (8).h5'          Untitled1.ipynb


In [6]:
# Solving for residual std scaling issue
import os
import math
import time
import inspect
from dataclasses import dataclass
import torch
import torch.nn as nn
from torch.nn import functional as F


class CausalSelfAttention(nn.Module):

    def __init__(self, config):
        super().__init__()
        assert config.n_embd % config.n_head == 0
        # key, query, value projections for all heads, but in a batch
        self.c_attn = nn.Linear(config.n_embd, 3 * config.n_embd)
        # output projection
        self.c_proj = nn.Linear(config.n_embd, config.n_embd)
        self.attn_dropout = nn.Dropout(config.dropout)
        self.resid_dropout = nn.Dropout(config.dropout)
        # regularization
        self.n_head = config.n_head
        self.n_embd = config.n_embd
        self.dropout = config.dropout
        self.register_buffer(
            "bias",
            torch.tril(torch.ones(config.block_size, config.block_size)).view(
                1, 1, config.block_size, config.block_size
            ),
        )

    def forward(self, x):
        B, T, C = (
            x.size()
        )  # batch size, sequence length, embedding dimensionality (n_embd)
        # calculate query, key, values for all heads in batch and move head forward to be the batch dim
        # nh is "number of heads", hs is "head size", and C (number of channels) = nh * hs
        # e.g. in GPT-2 (124M), n_head=12, hs=64, so nh*hs=C=768 channels in the Transformer
        qkv = self.c_attn(x)
        q, k, v = qkv.split(self.n_embd, dim=2)
        k = k.view(B, T, self.n_head, C // self.n_head).transpose(
            1, 2
        )  # (B, nh, T, hs)
        q = q.view(B, T, self.n_head, C // self.n_head).transpose(
            1, 2
        )  # (B, nh, T, hs)
        v = v.view(B, T, self.n_head, C // self.n_head).transpose(
            1, 2
        )  # (B, nh, T, hs)

        att = (q @ k.transpose(-2, -1)) * (1.0 / math.sqrt(k.size(-1)))
        att = att.masked_fill(self.bias[:, :, :T, :T] == 0, float("-inf"))
        att = F.softmax(att, dim=-1)
        att = self.attn_dropout(att)
        y = att @ v  # (B, nh, T, T) x (B, nh, T, hs) -> (B, nh, T, hs)

        y = (
            y.transpose(1, 2).contiguous().view(B, T, C)
        )  # re-assemble all head outputs side by side
        # output projection
        y = self.resid_dropout(self.c_proj(y))
        return y


class MLP(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.c_fc = nn.Linear(config.n_embd, 4 * config.n_embd)
        self.gelu = nn.GELU(approximate="tanh")
        self.c_proj = nn.Linear(4 * config.n_embd, config.n_embd)
        self.c_proj.NANOGPT_SCALE_INIT = 1

    def forward(self, x):
        x = self.c_fc(x)
        x = self.gelu(x)
        x = self.c_proj(x)
        return x


class Block(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.ln_1 = nn.LayerNorm(config.n_embd)
        self.attn = CausalSelfAttention(config)
        self.ln_2 = nn.LayerNorm(config.n_embd)
        self.mlp = MLP(config)

    def forward(self, x):
        x = x + self.attn(self.ln_1(x))
        x = x + self.mlp(self.ln_2(x))
        return x


@dataclass
class GPTConfig:
    block_size: int = 1024  # Reduced from 1024 to save memory
    vocab_size: int = 50257
    n_layer: int = 12  # Reduced from 12
    n_head: int = 12  # Reduced from 12
    n_embd: int = 768  # Reduced from 768
    dropout: float = 0.1  # Added dropout for regularization


class GPT(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.config = config

        self.transformer = nn.ModuleDict(
            dict(
                wte=nn.Embedding(config.vocab_size, config.n_embd),
                wpe=nn.Embedding(config.block_size, config.n_embd),
                h=nn.ModuleList([Block(config) for _ in range(config.n_layer)]),
                ln_f=nn.LayerNorm(config.n_embd),
            )
        )
        self.lm_head = nn.Linear(config.n_embd, config.vocab_size, bias=False)

        # weight sharing
        self.transformer.wte.weight = self.lm_head.weight

        # weight initialization
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            std = 0.02
            if hasattr(module, "NANGPT_SCALE_INIT"):
                std *= (2 * self.config.n_layer) ** -0.5
            torch.nn.init.normal_(module.weight, mean=0.0, std=std)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        # idx is of shape (B, T)
        B, T = idx.size()
        assert (
            T <= self.config.block_size
        ), f"Cannot forward sequence of length {T}, block size is only {self.config.block_size}"
        # forward the token and posisition embeddings
        pos = torch.arange(0, T, dtype=torch.long, device=idx.device)  # shape (T)
        pos_emb = self.transformer.wpe(pos)  # position embeddings of shape (T, n_embd)
        tok_emb = self.transformer.wte(idx)  # token embeddings of shape (B, T, n_embd)
        x = tok_emb + pos_emb
        # forward the blocks of the transformer
        for block in self.transformer.h:
            x = block(x)
        # forward the final layernorm and the classifier
        x = self.transformer.ln_f(x)
        logits = self.lm_head(x)  # (B, T, vocab_size)
        loss = None
        if targets is not None:
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1))
        return logits, loss

    @classmethod
    def from_pretrained(cls, model_type):
        """Loads pretrained GPT-2 model weights from huggingface"""
        assert model_type in {"gpt2", "gpt2-medium", "gpt2-large", "gpt2-xl"}
        from transformers import GPT2LMHeadModel

        print("loading weights from pretrained gpt: %s" % model_type)

        # n_layer, n_head and n_embd are determined from model_type
        config_args = {
            "gpt2": dict(n_layer=12, n_head=12, n_embd=768),  # 124M params
            "gpt2-medium": dict(n_layer=24, n_head=16, n_embd=1024),  # 350M params
            "gpt2-large": dict(n_layer=36, n_head=20, n_embd=1280),  # 774M params
            "gpt2-xl": dict(n_layer=48, n_head=25, n_embd=1600),  # 1558M params
        }[model_type]
        config_args["vocab_size"] = 50257  # always 50257 for GPT model checkpoints
        config_args["block_size"] = 1024  # always 1024 for GPT model checkpoints
        # create a from-scratch initialized minGPT model
        config = GPTConfig(**config_args)
        model = GPT(config)
        sd = model.state_dict()
        sd_keys = sd.keys()
        sd_keys = [
            k for k in sd_keys if not k.endswith(".attn.bias")
        ]  # discard this mask / buffer, not a param

        # init a huggingface/transformers model
        model_hf = GPT2LMHeadModel.from_pretrained(model_type)
        sd_hf = model_hf.state_dict()

        # copy while ensuring all of the parameters are aligned and match in names and shapes
        sd_keys_hf = sd_hf.keys()
        sd_keys_hf = [
            k for k in sd_keys_hf if not k.endswith(".attn.masked_bias")
        ]  # ignore these, just a buffer
        sd_keys_hf = [
            k for k in sd_keys_hf if not k.endswith(".attn.bias")
        ]  # same, just the mask (buffer)
        transposed = [
            "attn.c_attn.weight",
            "attn.c_proj.weight",
            "mlp.c_fc.weight",
            "mlp.c_proj.weight",
        ]
        # basically the openai checkpoints use a "Conv1D" module, but we only want to use a vanilla Linear
        # this means that we have to transpose these weights when we import them
        assert len(sd_keys_hf) == len(
            sd_keys
        ), f"mismatched keys: {len(sd_keys_hf)} != {len(sd_keys)}"
        for k in sd_keys_hf:
            if any(k.endswith(w) for w in transposed):
                # special treatment for the Conv1D weights we need to transpose
                assert sd_hf[k].shape[::-1] == sd[k].shape
                with torch.no_grad():
                    sd[k].copy_(sd_hf[k].t())
            else:
                # vanilla copy over the other parameters
                assert sd_hf[k].shape == sd[k].shape
                with torch.no_grad():
                    sd[k].copy_(sd_hf[k])

        return model


# model = GPT.from_pretrained('gpt2')

device = "cpu"
if torch.cuda.is_available():
    device = "cuda"
elif hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
    device = "mps"
print(f"using device: {device}")

# SEED
torch.manual_seed(1337)
if torch.cuda.is_available():
    torch.cuda.manual_seed(1337)

# STOP
num_return_sequences = 5
max_length = 30


import tiktoken


class DataLoaderLite:
    def __init__(self, B, T):
        self.B = B
        self.T = T

        # at init load tokens from disk and store them in memory
        with open("/content/drive/MyDrive/transformer_input/input.txt", "r") as f:
            text = f.read()
        enc = tiktoken.get_encoding("gpt2")
        tokens = enc.encode(text)
        self.tokens = torch.tensor(tokens)
        print(f"loaded {len(self.tokens)} tokens")
        print(f"1 epoch = {len(self.tokens) // (B * T)} batches")

        # state
        self.current_position = 0

    def next_batch(self):
        B, T = self.B, self.T
        buf = self.tokens[self.current_position : self.current_position + B * T + 1]
        x = (buf[:-1]).view(B, T)  # inputs
        y = (buf[1:]).view(B, T)  # targets
        # advance the position in the tensor
        self.current_position += B * T
        # if loading the next batch would be out of bounds, reset
        if self.current_position + (B * T + 1) > len(self.tokens):
            self.current_position = 0
        return x, y


model = GPT(GPTConfig())
model.to(device)


# Print model size
def count_parameters(model):
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    param_size = sum(p.numel() * p.element_size() for p in model.parameters()) / (
        1024 * 1024
    )  # Size in MB

    print(f"Total Parameters: {total_params:,}")
    print(f"Trainable Parameters: {trainable_params:,}")
    print(f"Model Size: {param_size:.2f} MB")

    # Print size by layer type
    layer_sizes = {}
    for name, p in model.named_parameters():
        layer_type = name.split(".")[0]
        size = p.numel() * p.element_size() / (1024 * 1024)  # Size in MB
        layer_sizes[layer_type] = layer_sizes.get(layer_type, 0) + size

    print("\nSize by component:")
    for layer_type, size in layer_sizes.items():
        print(f"{layer_type}: {size:.2f} MB")


count_parameters(model)

# Increase batch size and reduce sequence length
train_loader = DataLoaderLite(B=16, T=128)

# Calculate total steps
total_steps = 20000  # Increased from 10000
print(f"Training for {total_steps} steps")

# Initialize optimizer with better parameters
optimizer = torch.optim.AdamW(
    model.parameters(), lr=2e-4, weight_decay=0.1, betas=(0.9, 0.95), eps=1e-8
)

# Use OneCycleLR scheduler with better parameters
scheduler = torch.optim.lr_scheduler.OneCycleLR(
    optimizer,
    max_lr=2e-4,
    total_steps=total_steps,
    pct_start=0.1,
    anneal_strategy="cos",
    cycle_momentum=False,
    div_factor=10.0,
    final_div_factor=50.0,
)

# Training loop
best_loss = float("inf")
step = 0
losses = []  # Keep track of losses for monitoring
last_time = time.time()
interval = 100  # Print every 10 steps
moving_avg_size = 100  # Window size for moving average

while step < total_steps:
    x, y = train_loader.next_batch()
    x, y = x.to(device), y.to(device)

    optimizer.zero_grad()
    logits, loss = model(x, y)
    loss.backward()

    # Gradient clipping
    torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)  # Reduced from 1.0

    optimizer.step()
    scheduler.step()

    # Update best loss
    if loss.item() < best_loss:
        best_loss = loss.item()

    losses.append(loss.item())

    # Calculate moving average loss
    avg_loss = sum(losses[-moving_avg_size:]) / min(len(losses), moving_avg_size)

    # Print progress
    if step % interval == 0:
        current_time = time.time()
        time_per_batch = (current_time - last_time) / interval if step > 0 else 0
        last_time = current_time

        print(
            f"step {step}, "
            f"loss: {loss.item():.4f}, "
            f"avg_loss: {avg_loss:.4f}, "
            f"best_loss: {best_loss:.4f}, "
            f"lr: {scheduler.get_last_lr()[0]:.2e}, "
            f"time/batch: {time_per_batch:.3f}s"
        )

    # Check if moving average loss is below threshold
    if avg_loss < 0.099999:
        print(f"\nReached target average loss of {avg_loss:.6f} at step {step}")
        break

    step += 1

print(f"Final loss: {loss.item():.6f}")
print(f"Best loss: {best_loss:.6f}")
print(f"Final average loss: {avg_loss:.6f}")

# Save the trained model with compression
save_path = "trained_model.pt"
torch.save(
    {
        "model_state_dict": model.state_dict(),
        "config": model.config,
    },
    save_path,
)
print(f"Model saved to {save_path}")

# Generation code
enc = tiktoken.get_encoding("gpt2")
prompt = "We are accounted poor citizens, the"
tokens = enc.encode(prompt)
tokens = torch.tensor(tokens, dtype=torch.long)
tokens = tokens.unsqueeze(0).repeat(num_return_sequences, 1)
x = tokens.to(device)

using device: cuda
Total Parameters: 124,439,808
Trainable Parameters: 124,439,808
Model Size: 474.70 MB

Size by component:
transformer: 474.70 MB
loaded 338025 tokens
1 epoch = 165 batches
Training for 20000 steps
step 0, loss: 10.9502, avg_loss: 10.9502, best_loss: 10.9502, lr: 2.00e-05, time/batch: 0.000s
step 100, loss: 7.3847, avg_loss: 8.3567, best_loss: 7.3148, lr: 2.11e-05, time/batch: 0.493s
step 200, loss: 6.1754, avg_loss: 6.8560, best_loss: 6.1491, lr: 2.45e-05, time/batch: 0.525s
step 300, loss: 5.9571, avg_loss: 6.2103, best_loss: 5.6249, lr: 2.99e-05, time/batch: 0.518s
step 400, loss: 5.5309, avg_loss: 5.8262, best_loss: 5.3845, lr: 3.73e-05, time/batch: 0.522s
step 500, loss: 5.6939, avg_loss: 5.6099, best_loss: 4.8133, lr: 4.65e-05, time/batch: 0.523s
step 600, loss: 5.5644, avg_loss: 5.3130, best_loss: 4.7059, lr: 5.72e-05, time/batch: 0.525s
step 700, loss: 5.1696, avg_loss: 5.1070, best_loss: 4.2967, lr: 6.93e-05, time/batch: 0.525s
step 800, loss: 4.3263, avg_los